In [11]:
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
from rasterio.mask import mask
from rasterio.features import rasterize

In [12]:
def classify_flood_depth(depth_array, thresholds=None):
    """
    Classify flood depths into categories based on custom thresholds
    
    Parameters:
    depth_array: numpy array of flood depth values
    thresholds: list of threshold values (e.g., [0.5, 1.0, 2.0, 3.0])
                If None, uses default thresholds
    
    Returns:
    classified array where:
    0 = No flood (depth = 0)
    1 = First category (0 < depth <= thresholds[0])
    2 = Second category (thresholds[0] < depth <= thresholds[1])
    ... and so on
    """
    # Default thresholds if none provided
    if thresholds is None:
        thresholds = [0.5, 1.0, 2.0, 3.0]
    
    classified = np.zeros_like(depth_array)
    
    # No flood category
    classified[depth_array == 0] = 0
    
    # Dynamic classification based on thresholds
    for i, threshold in enumerate(thresholds):
        if i == 0:
            # First category: 0 < depth <= first_threshold
            classified[(depth_array > 0) & (depth_array <= threshold)] = i + 1
        else:
            # Subsequent categories: previous_threshold < depth <= current_threshold
            classified[(depth_array > thresholds[i-1]) & (depth_array <= threshold)] = i + 1
    
    # Last category: depth > last_threshold
    classified[depth_array > thresholds[-1]] = len(thresholds) + 1
    
    return classified

In [27]:
def calculate_flood_area_by_village(flood_raster_path, villages_shapefile_path, thresholds=None, unit='m2'):
    """
    Calculate flood area by village with dynamic thresholds
    
    Parameters:
    flood_raster_path: path to flood depth raster
    villages_shapefile_path: path to villages shapefile
    thresholds: list of depth thresholds (e.g., [0.5, 1.0, 2.0, 3.0])
    unit: output unit for area calculation ('m2', 'km2', 'hectare', 'rai')
    """
    # Default thresholds if none provided
    if thresholds is None:
        thresholds = [0.5, 1.0, 2.0, 3.0]
    
    # Load data
    villages_gdf = gpd.read_file(villages_shapefile_path)
    
    with rasterio.open(flood_raster_path) as src:
        flood_data = src.read(1)
        transform = src.transform
        crs = src.crs
        
        # Calculate pixel area in square meters
        pixel_area_m2 = abs(transform.a * transform.e)  # in map units²
        
        # Convert to desired unit
        if unit == 'm2':
            pixel_area = pixel_area_m2
            unit_suffix = ' (m²)'
        elif unit == 'km2':
            pixel_area = pixel_area_m2 / 1_000_000  # Convert m² to km²
            unit_suffix = ' (km²)'
        elif unit == 'hectare':
            pixel_area = pixel_area_m2 / 10_000  # Convert m² to hectares
            unit_suffix = ' (hectares)'
        elif unit == 'rai':
            pixel_area = pixel_area_m2 / 1_600  # Convert m² to rai (1 rai = 1,600 m²)
            unit_suffix = ' (rai)'
        else:
            raise ValueError("Unit must be one of: 'm2', 'km2', 'hectare', 'rai'")
        
        results = []
        
        for idx, village in villages_gdf.iterrows():
            # Mask raster to village boundary
            village_geom = [village.geometry]
            masked_flood, masked_transform = mask(src, village_geom, crop=True)
            masked_classified = classify_flood_depth(masked_flood[0], thresholds)
            
            # Build dynamic result dictionary
            village_result = {
                'village_id': village.get('ADM3_EN', idx),
                'village_name': village.get('ADM3_TH', f'Village_{idx}'),
                f'total_area{unit_suffix}': np.sum(masked_classified >= 0) * pixel_area,
                f'no_flood_area{unit_suffix}': np.sum(masked_classified == 0) * pixel_area,
            }
            
            # Add dynamic threshold-based fields
            for i, threshold in enumerate(thresholds):
                if i == 0:
                    field_name = f'0-{threshold} m{unit_suffix}'
                else:
                    field_name = f'{thresholds[i-1]}-{threshold}m{unit_suffix}'
                village_result[field_name] = np.sum(masked_classified == i + 1) * pixel_area
            
            # Add final category (above last threshold)
            village_result[f'>{thresholds[-1]} m{unit_suffix}'] = np.sum(masked_classified == len(thresholds) + 1) * pixel_area
            
            # Total flooded area
            village_result[f'total_flooded_area{unit_suffix}'] = np.sum(masked_classified > 0) * pixel_area
            
            results.append(village_result)
    
    return pd.DataFrame(results)

In [28]:
# Usage examples:

# Calculate areas in rai (Thai unit)
custom_thresholds = [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2]
results_df_rai = calculate_flood_area_by_village('WD-Tr2.tif', 'Select_tambon_UTM48.shp', 
                                                custom_thresholds, unit='rai')

# # Calculate areas in square meters (default)
# results_df_m2 = calculate_flood_area_by_village('WD-Tr2.tif', 'Select_tambon_UTM48.shp', 
#                                                custom_thresholds, unit='m2')

# # Calculate areas in hectares
# results_df_hectare = calculate_flood_area_by_village('WD-Tr2.tif', 'Select_tambon_UTM48.shp', 
#                                                     custom_thresholds, unit='hectare')

# # Calculate areas in square kilometers
# results_df_km2 = calculate_flood_area_by_village('WD-Tr2.tif', 'Select_tambon_UTM48.shp', 
#                                                 custom_thresholds, unit='km2')

# Save results
results_df_rai.to_csv('flood_analysis_results_rai.csv', index=False)
print("Results calculated in rai:")
print(results_df_rai.head())

Results calculated in rai:
   village_id village_name  total_area (rai)  no_flood_area (rai)  \
0  Nai Mueang      ในเมือง      33747.817880         33151.106679   
1    Hua Ruea      หัวเรือ      46311.849030         46311.849030   
2   Nong Khon      หนองขอน      84799.838682         82569.582324   
3      Pathum         ปทุม       5193.732915          5168.778145   
4    Kham Yai      ขามใหญ่      56362.698925         56263.786156   

   0-0.1 m (rai)  0.1-0.25m (rai)  0.25-0.5m (rai)  0.5-0.75m (rai)  \
0      22.251466        19.860684        28.408122        24.345354   
1       0.000000         0.000000         0.000000         0.000000   
2      53.956681       104.538140       271.705407       379.509386   
3       2.031384         3.109580         7.922397         3.343970   
4       6.062900         4.578427        10.031911        28.126854   

   0.75-1.0m (rai)  1.0-1.5m (rai)  1.5-2m (rai)  >2 m (rai)  \
0        25.111030       68.285750     97.475174  310.973620   
1  

In [29]:
results_df_rai

,village_id,village_name,total_area (rai),no_flood_area (rai),0-0.1 m (rai),0.1-0.25m (rai),0.25-0.5m (rai),0.5-0.75m (rai),0.75-1.0m (rai),1.0-1.5m (rai),1.5-2m (rai),>2 m (rai),total_flooded_area (rai)
0,Nai Mueang,ในเมือง,33747.817880,33151.106679,22.251466,19.860684,28.408122,24.345354,25.111030,68.285750,97.475174,310.973620,596.711201
1,Hua Ruea,หัวเรือ,46311.849030,46311.849030,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Nong Khon,หนองขอน,84799.838682,82569.582324,53.956681,104.538140,271.705407,379.509386,350.569979,368.321149,311.833051,389.822566,2230.256359
3,Pathum,ปทุม,5193.732915,5168.778145,2.031384,3.109580,7.922397,3.343970,2.718929,3.765873,0.859432,1.203204,24.954770
4,Kham Yai,ขามใหญ่,56362.698925,56263.786156,6.062900,4.578427,10.031911,28.126854,12.250807,16.313575,10.172545,11.375750,98.912769
5,Chaeramae,แจระแม,52568.652005,46364.524376,177.761715,339.850523,748.611837,954.719170,783.317249,888.371048,635.979414,1675.516674,6204.127629
6,Nong Bo,หนองบ่อ,82511.687883,61918.908854,468.421496,747.049234,1516.474942,2067.261242,2511.462391,5433.029936,3110.517497,4738.562292,20592.779030
7,Rai Noi,ไร่น้อย,97609.557899,96832.022281,7.375486,14.516582,30.970791,44.096656,67.270058,216.358009,209.420052,187.527984,777.535618
8,Krasop,กระโสบ,63425.430015,62392.877590,12.844597,23.110898,52.722225,88.458955,123.086237,258.735802,248.500752,225.092960,1032.552425
9,Kut Lat,กุดลาด,89963.929019,82483.436021,161.854417,246.953775,456.951990,529.816167,651.949216,1320.462024,1359.402091,2753.103317,7480.492997


In [ ]:
results_df.to_csv('flood_analysis_results.csv', index=False)

In [8]:
# Usage
results_df = calculate_flood_area_by_village('WD-Tr2.tif', 'Select_tambon_UTM48.shp')
results_df.to_csv('flood_analysis_results.csv', index=False)